In [1]:
# Use requests library to download the page
# Use BS4 to parse and extract the info
# convert to pandas dataframe

Here are the steps we ae going to follow:

* We're going to scrape "https://github.com/topics"
* We'll get a list of topicss. For each topic, we'll get topic title, topic page URL and topic description
* For each topic, we'll get the top 25 repositories in the topic from topic page
* For each repo we will grab the repo name, username, stars and repo URL
* For each topic we'll create a CSV file in thr following format

repo_name, username, stars, repo_url


In [2]:
from bs4 import BeautifulSoup as bts


In [3]:
import requests


In [4]:
topics_url = r'https://github.com/topics'
r = requests.get(topics_url)

In [5]:
r.status_code


200

In [6]:
len(r.text)

165753

In [7]:
content = r.text

In [8]:
content[:1000]

'\n\n<!DOCTYPE html>\n<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"  data-a11y-animated-images="system" data-a11y-link-underlines="true">\n\n\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n  \n\n  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-a09cef873428.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-5d486a4ede8e.css" /><link data-color-theme="dark_dimmed" crossorigin="anonymous" med

In [9]:
#open html

# with open('webpage.html', 'w') as f:
#     f.write(page_content)

In [10]:
doc = bts(content,'html.parser')

In [11]:
doc.prettify()

'<!DOCTYPE html>\n<html data-a11y-animated-images="system" data-a11y-link-underlines="true" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <link href="https://github.githubassets.com" rel="dns-prefetch"/>\n  <link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>\n  <link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>\n  <link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>\n  <link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>\n  <link href="https://avatars.githubusercontent.com" rel="preconnect"/>\n  <link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-a09cef873428.css" media="all" rel="stylesheet">\n   <link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-5d486a4ede8e.css" media="all" rel="stylesheet">\n    <link crossorigin="anonymous" data-color-theme="dark_dimmed" data-h

In [12]:
selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
topic_title_tags = doc.find_all('p',{'class': selection_class})
len(topic_title_tags)

30

In [13]:
topic_title_tags[:5]

[<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ajax</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Algorithm</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Amp</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Android</p>]

In [14]:
topic_titles = []

for tag in topic_title_tags:
    topic_titles.append(tag.text)
print(topic_titles)    

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android', 'Angular', 'Ansible', 'API', 'Arduino', 'ASP.NET', 'Atom', 'Awesome Lists', 'Amazon Web Services', 'Azure', 'Babel', 'Bash', 'Bitcoin', 'Bootstrap', 'Bot', 'C', 'Chrome', 'Chrome extension', 'Command line interface', 'Clojure', 'Code quality', 'Code review', 'Compiler', 'Continuous integration', 'COVID-19', 'C++']


In [15]:
desc_selector = 'f5 color-fg-muted mb-0 mt-1'
topic_desc_tags = doc.find_all('p',{'class': desc_selector})

In [16]:
topic_desc_tags[:5]

[<p class="f5 color-fg-muted mb-0 mt-1">
           3D refers to the use of three-dimensional graphics, modeling, and animation in various industries.
         </p>,
 <p class="f5 color-fg-muted mb-0 mt-1">
           Ajax is a technique for creating interactive web applications.
         </p>,
 <p class="f5 color-fg-muted mb-0 mt-1">
           Algorithms are self-contained sequences that carry out a variety of tasks.
         </p>,
 <p class="f5 color-fg-muted mb-0 mt-1">
           Amp is a non-blocking concurrency library for PHP.
         </p>,
 <p class="f5 color-fg-muted mb-0 mt-1">
           Android is an operating system built by Google designed for mobile devices.
         </p>]

In [17]:
topic_descs = []

for tag in topic_desc_tags:
    topic_descs.append(tag.text.strip())
print(topic_descs[:10])    

['3D refers to the use of three-dimensional graphics, modeling, and animation in various industries.', 'Ajax is a technique for creating interactive web applications.', 'Algorithms are self-contained sequences that carry out a variety of tasks.', 'Amp is a non-blocking concurrency library for PHP.', 'Android is an operating system built by Google designed for mobile devices.', 'Angular is an open source web application platform.', 'Ansible is a simple and powerful automation engine.', 'An API (Application Programming Interface) is a collection of protocols and subroutines for building software.', 'Arduino is an open source platform for building electronic devices.', 'ASP.NET is a web framework for building modern web apps and services.']


In [18]:
topic_title_tags1 = topic_title_tags[1]

In [19]:
topic_title_tags1

<p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ajax</p>

In [20]:
topic_title_tags1.parent

<a class="no-underline flex-1 d-flex flex-column" href="/topics/ajax">
<p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ajax</p>
<p class="f5 color-fg-muted mb-0 mt-1">
          Ajax is a technique for creating interactive web applications.
        </p>
</a>

In [21]:
link_selector = "no-underline flex-grow-0"
topic_link_tags = doc.find_all('a',{'class': link_selector})

In [22]:
topic_link_tags

[<a class="no-underline flex-grow-0" href="/topics/3d">
 <div class="color-bg-accent f4 color-fg-muted text-bold rounded mr-3 flex-shrink-0 text-center" style="width:64px; height:64px; line-height:64px;">
             #
           </div>
 </a>,
 <a class="no-underline flex-grow-0" href="/topics/ajax">
 <img alt="ajax" class="rounded mr-3" height="64" src="https://raw.githubusercontent.com/github/explore/8be26d91eb231fec0b8856359979ac09f27173fd/topics/ajax/ajax.png" width="64"/>
 </a>,
 <a class="no-underline flex-grow-0" href="/topics/algorithm">
 <div class="color-bg-accent f4 color-fg-muted text-bold rounded mr-3 flex-shrink-0 text-center" style="width:64px; height:64px; line-height:64px;">
             #
           </div>
 </a>,
 <a class="no-underline flex-grow-0" href="/topics/amphp">
 <img alt="amphp" class="rounded mr-3" height="64" src="https://raw.githubusercontent.com/github/explore/99fe59c0f4fb5d6545311440b4ce89a0d82b0804/topics/amphp/amphp.png" width="64"/>
 </a>,
 <a class

In [23]:
len(topic_link_tags)

30

In [24]:
topic_link_tags[1]['href']

'/topics/ajax'

In [25]:
topic1_url = 'https://github.com' + topic_link_tags[1]['href']

In [26]:
print(topic1_url)

https://github.com/topics/ajax


In [27]:
topics_urls = []
base_url = 'https://github.com'

for url in topic_link_tags:
    topics_urls.append(base_url + url['href'])
    
for i in topics_urls:
    print('\n',i)    


 https://github.com/topics/3d

 https://github.com/topics/ajax

 https://github.com/topics/algorithm

 https://github.com/topics/amphp

 https://github.com/topics/android

 https://github.com/topics/angular

 https://github.com/topics/ansible

 https://github.com/topics/api

 https://github.com/topics/arduino

 https://github.com/topics/aspnet

 https://github.com/topics/atom

 https://github.com/topics/awesome

 https://github.com/topics/aws

 https://github.com/topics/azure

 https://github.com/topics/babel

 https://github.com/topics/bash

 https://github.com/topics/bitcoin

 https://github.com/topics/bootstrap

 https://github.com/topics/bot

 https://github.com/topics/c

 https://github.com/topics/chrome

 https://github.com/topics/chrome-extension

 https://github.com/topics/cli

 https://github.com/topics/clojure

 https://github.com/topics/code-quality

 https://github.com/topics/code-review

 https://github.com/topics/compiler

 https://github.com/topics/continuous-integratio

In [28]:
import pandas as pd

In [29]:
topics_dict = {
    'title': topic_titles,
    'description': topic_descs,
    'url': topics_urls
}

In [30]:
topic_df = pd.DataFrame(topics_dict)

In [31]:
topic_df.head()

,title,description,url
0,3D,3D refers to the use of three-dimensional grap...,https://github.com/topics/3d
1,Ajax,Ajax is a technique for creating interactive w...,https://github.com/topics/ajax
2,Algorithm,Algorithms are self-contained sequences that c...,https://github.com/topics/algorithm
3,Amp,Amp is a non-blocking concurrency library for ...,https://github.com/topics/amphp
4,Android,Android is an operating system built by Google...,https://github.com/topics/android


In [32]:
# Get information from topic page url

In [33]:
topic_page_url = topics_urls[1]
topic_page_url

'https://github.com/topics/ajax'

In [34]:
topic_page_url = topics_urls[1]
r = requests.get(topic_page_url)
topics_doc  = bts(r.text,'html.parser')

In [35]:
r

<Response [200]>

In [36]:
len(r.text)

442909

In [37]:
topics_doc  = bts(r.text,'html.parser')
topics_doc


<!DOCTYPE html>

<html data-a11y-animated-images="system" data-a11y-link-underlines="true" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
<link href="https://avatars.githubusercontent.com" rel="preconnect"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-a09cef873428.css" media="all" rel="stylesheet"><link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-5d486a4ede8e.css" media="all" rel="stylesheet"><link crossorigin="anonymous" data-color-theme="dark_dimmed" data-href="https://github.githubassets.com/

In [38]:
h3_selection_class = 'f3 color-fg-muted text-normal lh-condensed'
repo_tags = topics_doc.find_all('h3',{'class':h3_selection_class})


In [39]:
repo_tags

[<h3 class="f3 color-fg-muted text-normal lh-condensed">
 <a class="Link" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":36322912,"originating_url":"https://github.com/topics/ajax","user_id":null}}' data-hydro-click-hmac="9074acd8ed677416751ed4a6a8764f616627f58efec6a83519e500c62fe3c738" data-turbo="false" data-view-component="true" href="/ljianshu">
             ljianshu
 </a>          /
           <a class="Link text-bold wb-break-word" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":137582912,"originating_url":"https://github.com/topics/ajax","user_id":null}}' data-hydro-click-hmac="5f285461b513fd5a20b16ea1e5d33b4a0a01a278ae2357647e16b2f4938e9dfc" data-turbo="false" data-view-co

In [40]:
len(repo_tags)

20

In [41]:
repo_tags[0]

<h3 class="f3 color-fg-muted text-normal lh-condensed">
<a class="Link" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":36322912,"originating_url":"https://github.com/topics/ajax","user_id":null}}' data-hydro-click-hmac="9074acd8ed677416751ed4a6a8764f616627f58efec6a83519e500c62fe3c738" data-turbo="false" data-view-component="true" href="/ljianshu">
            ljianshu
</a>          /
          <a class="Link text-bold wb-break-word" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":137582912,"originating_url":"https://github.com/topics/ajax","user_id":null}}' data-hydro-click-hmac="5f285461b513fd5a20b16ea1e5d33b4a0a01a278ae2357647e16b2f4938e9dfc" data-turbo="false" data-view-compone

In [42]:
repo_tags[0].text

'\n\n            ljianshu\n          /\n          \n            Blog\n '

In [43]:
repo_tags[2].text

'\n\n            olifolkerd\n          /\n          \n            tabulator\n '

In [44]:
a_tags = repo_tags[0].find_all('a')


In [45]:
a_tags

[<a class="Link" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":36322912,"originating_url":"https://github.com/topics/ajax","user_id":null}}' data-hydro-click-hmac="9074acd8ed677416751ed4a6a8764f616627f58efec6a83519e500c62fe3c738" data-turbo="false" data-view-component="true" href="/ljianshu">
             ljianshu
 </a>,
 <a class="Link text-bold wb-break-word" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":137582912,"originating_url":"https://github.com/topics/ajax","user_id":null}}' data-hydro-click-hmac="5f285461b513fd5a20b16ea1e5d33b4a0a01a278ae2357647e16b2f4938e9dfc" data-turbo="false" data-view-component="true" href="/ljianshu/Blog">
             Blog
 </a>]

In [46]:
a_tags[0].text.strip()

'ljianshu'

In [47]:
a_tags[1].text.strip()

'Blog'

In [48]:
#url
a_tags[1]['href']

'/ljianshu/Blog'

In [49]:
base_url + a_tags[1]['href']

'https://github.com/ljianshu/Blog'

In [50]:
star_tags = topics_doc.find_all('span',{'class': 'Counter js-social-count'})

In [51]:
star_tags

[<span aria-label="7712 users starred this repository" class="Counter js-social-count" data-plural-suffix="users starred this repository" data-singular-suffix="user starred this repository" data-turbo-replace="true" data-view-component="true" id="repo-stars-counter-star" title="7,712">7.7k</span>,
 <span aria-label="7328 users starred this repository" class="Counter js-social-count" data-plural-suffix="users starred this repository" data-singular-suffix="user starred this repository" data-turbo-replace="true" data-view-component="true" id="repo-stars-counter-star" title="7,328">7.3k</span>,
 <span aria-label="5703 users starred this repository" class="Counter js-social-count" data-plural-suffix="users starred this repository" data-singular-suffix="user starred this repository" data-turbo-replace="true" data-view-component="true" id="repo-stars-counter-star" title="5,703">5.7k</span>,
 <span aria-label="5639 users starred this repository" class="Counter js-social-count" data-plural-suff

In [52]:
star_tags[0].text.strip()

'7.7k'

In [53]:
def parse_star_count(stars_str):
    stars_str = stars_str.strip()
    if stars_str[-1] == 'k':
        return int(float(stars_str[:-1])*1000)
    else:
        return int(stars_str)

In [54]:
parse_star_count(star_tags[0].text.strip())

7700

In [55]:
def get_repo_info(h3_tag,star_tag):
    print("function get_repo_info started")
    a_tag = h3_tag.find_all('a')
    username = a_tag[0].text.strip()
    repo_name = a_tag[1].text.strip()
    repo_url= base_url + a_tag[1]['href']
    stars = parse_star_count(star_tag.text.strip())
    return username, repo_name, repo_url, stars
    print(username,repo_name, repo_url, stars)
print("get_repo_info function completed")    

get_repo_info function completed


In [56]:
repo_tags[0]

<h3 class="f3 color-fg-muted text-normal lh-condensed">
<a class="Link" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":36322912,"originating_url":"https://github.com/topics/ajax","user_id":null}}' data-hydro-click-hmac="9074acd8ed677416751ed4a6a8764f616627f58efec6a83519e500c62fe3c738" data-turbo="false" data-view-component="true" href="/ljianshu">
            ljianshu
</a>          /
          <a class="Link text-bold wb-break-word" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":137582912,"originating_url":"https://github.com/topics/ajax","user_id":null}}' data-hydro-click-hmac="5f285461b513fd5a20b16ea1e5d33b4a0a01a278ae2357647e16b2f4938e9dfc" data-turbo="false" data-view-compone

In [57]:
repo_tags[2]

<h3 class="f3 color-fg-muted text-normal lh-condensed">
<a class="Link" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":6840524,"originating_url":"https://github.com/topics/ajax","user_id":null}}' data-hydro-click-hmac="be8e6f23f73e1abe2c958bad054c17f9545d23c2f56f898adb4bffe80e89011e" data-turbo="false" data-view-component="true" href="/olifolkerd">
            olifolkerd
</a>          /
          <a class="Link text-bold wb-break-word" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":44867148,"originating_url":"https://github.com/topics/ajax","user_id":null}}' data-hydro-click-hmac="773a01199d7467fcb0a03fb86f78130f0e2b6414b08e564685fb4dea216afb97" data-turbo="false" data-view-compo

In [58]:
get_repo_info(repo_tags[2],star_tags[2])

function get_repo_info started


('olifolkerd', 'tabulator', 'https://github.com/olifolkerd/tabulator', 5700)

In [59]:
topic_repos_dict = {
    'username' : [],
    'repo_name': [],
    'stars':[],
    'repo_url':[]
}

for i in range(len(repo_tags)):
    repo_info = get_repo_info(repo_tags[i],star_tags[i])
    topic_repos_dict['username'].append(repo_info[0])
    topic_repos_dict['repo_name'].append(repo_info[1])
    topic_repos_dict['repo_url'].append(repo_info[2])
    topic_repos_dict['stars'].append(repo_info[3])

function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started


In [60]:
topic_repos_df = pd.DataFrame(topic_repos_dict)
topic_repos_df

,username,repo_name,stars,repo_url
0,ljianshu,Blog,7700,https://github.com/ljianshu/Blog
1,metafizzy,infinite-scroll,7300,https://github.com/metafizzy/infinite-scroll
2,olifolkerd,tabulator,5700,https://github.com/olifolkerd/tabulator
3,developit,unfetch,5600,https://github.com/developit/unfetch
4,jquery-form,form,5200,https://github.com/jquery-form/form
5,Studio-42,elFinder,4500,https://github.com/Studio-42/elFinder
6,elbywan,wretch,4200,https://github.com/elbywan/wretch
7,dwyl,learn-to-send-email-via-google-script-html-no-...,3000,https://github.com/dwyl/learn-to-send-email-vi...
8,ded,reqwest,2900,https://github.com/ded/reqwest
9,wendux,ajax-hook,2400,https://github.com/wendux/ajax-hook


In [61]:
from bs4 import BeautifulSoup

In [62]:
def get_topic_repos(topic_url):
    response = requests.get(topic_url)
    if response.status_code != 200:
        raise Exception("Failed to load page{}".format(topic_url))
    topic_doc = bts(response.text,'html.parser')
    h3_selection_class = 'f3 color-fg-muted text-normal lh-condensed'
    repo_tags = topic_doc.find_all("h3",{'class' : h3_selection_class})
    star_tags = topic_doc.find_all('span',{'class':'Counter js-social-count'})
    
    
    topic_repos_dict = {
    'username' : [],
    'repo_name': [],
    'stars':[],
    'repo_url':[]
    }

    for i in range(len(repo_tags)):
        repo_info = get_repo_info(repo_tags[i],star_tags[i])
        topic_repos_dict['username'].append(repo_info[0])
        topic_repos_dict['repo_name'].append(repo_info[1])
        topic_repos_dict['stars'].append(repo_info[2])
        topic_repos_dict['repo_url'].append(repo_info[3])
    return pd.DataFrame(topic_repos_dict)

In [63]:
def get_topic_page(topic_url):
    response = requests.get(topic_url)
    if response.status_code != 200:
        raise Exception("Failed to load page{}".format(topic_url))
    doc = bts(response.text,'html_parser')
    return doc
    print(doc)
print('get_topic_page function completed')    

get_topic_page function completed


In [64]:
topics_urls[4]

'https://github.com/topics/android'

In [65]:
get_topic_repos(topics_urls[4])

function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started
function get_repo_info started


,username,repo_name,stars,repo_url
0,flutter,flutter,https://github.com/flutter/flutter,157000
1,facebook,react-native,https://github.com/facebook/react-native,112000
2,justjavac,free-programming-books-zh_CN,https://github.com/justjavac/free-programming-...,105000
3,Genymobile,scrcpy,https://github.com/Genymobile/scrcpy,91700
4,Hack-with-Github,Awesome-Hacking,https://github.com/Hack-with-Github/Awesome-Ha...,69600
5,google,material-design-icons,https://github.com/google/material-design-icons,48700
6,Solido,awesome-flutter,https://github.com/Solido/awesome-flutter,48600
7,wasabeef,awesome-android-ui,https://github.com/wasabeef/awesome-android-ui,47500
8,square,okhttp,https://github.com/square/okhttp,44500
9,android,architecture-samples,https://github.com/android/architecture-samples,43200


In [66]:
def scrape_topic(topic_url, path):
    print("Function scrape_topic started")
    if os.path.exists(path):
        print("The file {} already exists. Skipping...".format(path))
        return
    topic_df = get_topic_repos(get_topic_page(topic_url))
#     topic_df.to_csv(path, index=None)
    print(topic_df)
print("scrape_topic function completed")

scrape_topic function completed


In [67]:
pwd

'C:\\Users\\virat'

In [68]:
path = r'C://Users//virat/topic_data1'